In [26]:
# Install once only - python official library from AWS
# !pip install boto3

### define configuration

In [27]:
aws_accesskey = <Your Access Key>
aws_secretaccess = <Your secret access key>
myregion= <your region>

In [28]:
import boto3

In [29]:
def Detect_PPE(aws_access,aws_secret, aws_region,Image,Bucket_Name):
    Rekognition_client = boto3.client('rekognition',aws_access_key_id=aws_access,aws_secret_access_key=aws_secret,region_name =aws_region)
    response = Rekognition_client.detect_protective_equipment(Image={'S3Object': {'Bucket': Bucket_Name, 'Name': Image}},
                                                  SummarizationAttributes={'MinConfidence': 80,
                                                                           'RequiredEquipmentTypes': ['FACE_COVER',
                                                                                                      'HAND_COVER',
                                                                                                      'HEAD_COVER']})
    print('Detected PPE for people in image ' + Image)
    print('\nDetected people\n---------------')
    for person in response['Persons']:
        print('Person ID: ' + str(person['Id']))
        print('Body Parts\n----------')
        body_parts = person['BodyParts']
        if len(body_parts) == 0:
            print('No body parts found')
        else:
            for body_part in body_parts:
                print('\t' + body_part['Name'] + '\n\t\tConfidence: ' + str(body_part['Confidence']))
                print('\n\t\tDetected PPE\n\t\t------------')
                ppe_items = body_part['EquipmentDetections']
                if len(ppe_items) == 0:
                     print('\t\tNo PPE detected on ' + body_part['Name'])
                else:
                    for ppe_item in ppe_items:
                        print('\t\t' + ppe_item['Type'] + '\n\t\t\tConfidence: ' + str(ppe_item['Confidence']))
                        print('\t\tCovers body part: ' + str(ppe_item['CoversBodyPart']['Value']) + '\n\t\t\tConfidence: ' + str(
                            ppe_item['CoversBodyPart']['Confidence']))
                        print('\t\tBounding Box:')
                        print('\t\t\tTop: ' + str(ppe_item['BoundingBox']['Top']))
                        print('\t\t\tLeft: ' + str(ppe_item['BoundingBox']['Left']))
                        print('\t\t\tWidth: ' + str(ppe_item['BoundingBox']['Width']))
                        print('\t\t\tHeight: ' + str(ppe_item['BoundingBox']['Height']))
                        print('\t\t\tConfidence: ' + str(ppe_item['Confidence']))
            print()
        print()
    print('Person ID Summary\n----------------')
    display_summary('With required equipment', response['Summary']['PersonsWithRequiredEquipment'])
    display_summary('Without required equipment', response['Summary']['PersonsWithoutRequiredEquipment'])
    display_summary('Indeterminate', response['Summary']['PersonsIndeterminate'])

    print()
    return len(response['Persons'])

# Display summary information for supplied summary.
def display_summary(summary_type, summary):
    print(summary_type + '\n\tIDs: ', end='')
    if (len(summary) == 0):
        print('None')
    else:
        for num, id in enumerate(summary, start=0):
            if num == len(summary) - 1:
                print(id)
            else:
                print(str(id) + ', ', end='')

In [30]:
Detect_PPE(aws_accesskey,aws_secretaccess,myregion,<the name of image>,<s3 bucket name>)

Detected PPE for people in image moving-needle-covid-blog-image-749x470.png

Detected people
---------------
Person ID: 0
Body Parts
----------
	FACE
		Confidence: 99.69596862792969

		Detected PPE
		------------
		FACE_COVER
			Confidence: 99.70165252685547
		Covers body part: True
			Confidence: 99.49976348876953
		Bounding Box:
			Top: 0.32367491722106934
			Left: 0.11036142706871033
			Width: 0.11230054497718811
			Height: 0.11004280298948288
			Confidence: 99.70165252685547
	HEAD
		Confidence: 99.9775619506836

		Detected PPE
		------------
		No PPE detected on HEAD


Person ID: 1
Body Parts
----------
	FACE
		Confidence: 99.83133697509766

		Detected PPE
		------------
		FACE_COVER
			Confidence: 99.98371887207031
		Covers body part: True
			Confidence: 75.3584976196289
		Bounding Box:
			Top: 0.5289948582649231
			Left: 0.5472688674926758
			Width: 0.10503917932510376
			Height: 0.11357937753200531
			Confidence: 99.98371887207031
	LEFT_HAND
		Confidence: 98.65900421142578

		De

3